In [1]:
#include <atomic>
#include <chrono>
#include <functional>
#include <iostream>
#include <set>
#include <thread>
#include <utility>
#include <vector>

In [2]:
using namespace std;

In [3]:
using namespace chrono;

In [4]:
struct temp {
    void set_property(int) { }
    void operation() { }
};

const int value = 42;
std::shared_ptr<temp> p = make_shared<temp>();

p->set_property(value);
// Someplace else...
p->set_property(value);


In [5]:
namespace v1 {

using type = temp;

void f(type* p) {
    //...
    p->operation();
    //
}

} // namespace v1

In [ ]:
namespace v1 {

using type = temp;

void f(type* p) {
    //...
    if (p) p->operation();
    //
}

} // namespace v1

In [ ]:
namespace v1 {

using type = temp;

void f(type& a) {
    //...
    a.operation();
    //
}

} // namespace v1

In [6]:
f(nullptr)

input_line_11:6:5: warning: null passed to a callee that requires a non-null argument [-Wnonnull]
    p->operation();
    ^


Interpreter Exception: 

Problem statement: Save documents every 5 minutes after application has been idle for 5 seconds.

Save documents every _n_ minutes after application has been idle for _m_ seconds.

**Save documents every _n_ minutes** after application has been idle for _m_ seconds.

**Save documents every _n_ minutes** - we can use a recurring timer.<sup>*</sup>

(Not really - but we'll come back to it.)

**Invoke a tast after the application has been idle for _m_ seconds.**

In [ ]:
?std::vector

In [ ]:
namespace v1 {

extern system_clock::time_point _last_idle;
void invoke_after(system_clock::duration, function<void()>);

template <class F> // F is task of the form void()
void after_idle(F task, system_clock::duration delay) {
    auto when = delay - (system_clock::now() - _last_idle);
    
    if (system_clock::duration::zero() < when) {
        invoke_after([=] { after_idle(task, delay); });
    } else {
        task();
    }
}

} // namespace v1

In [ ]:
namespace v2 {

template <class S, // S is a scheduler void(task, duration)
          class T, // T is a timer duration()
          class F> // F is a task void()
void on_expiration(S scheduler, T timer, F task) {
    struct local {
        static void recursive(S& scheduler, T timer, F task) {
            auto remaining = timer();
            if (remaining) {
                scheduler(remaining, [scheduler, _timer = move(timer),
                                      _task = move(task)]() mutable {
                    recursive(scheduler, move(_timer), move(_task));
                });
            } else {
                task();
            }
        }
    };
    auto remaining = timer(); // invoke before move
    scheduler(remaining,
              [scheduler, _timer = move(timer), _task = move(task)]() mutable {
                  local::recursive(scheduler, move(_timer), move(_task));
              });
}

} // namespace v2

In [ ]:
namespace v3 {

template <class S, class T, class F>
void recursive(S& scheduler, T timer, F task) {
    auto remaining = timer();
    if (decltype(remaining){0} < remaining) {
        scheduler(remaining, [scheduler, _timer = move(timer),
                              _task = move(task)]() mutable {
            recursive(scheduler, move(_timer), move(_task));
        });
    } else {
        task();
    }
}

} // namespace v3

In [ ]:
namespace v3 {

template <class S, // S is a scheduler void(task, duration)
          class T, // T is a timer duration()
          class F> // F is a task void()
void on_expiration(S scheduler, T timer, F task) {
    auto remaining = timer(); // invoke before move
    scheduler(remaining,
              [scheduler, _timer = move(timer), _task = move(task)]() mutable {
                  recursive(scheduler, move(_timer), move(_task));
              });
}

} // namespace v3

In [ ]:
#if 0
v3::on_expiration(
    [](auto after, auto task) {
        cout << "try again in " << duration_cast<duration<double>>(after).count() << " seconds." << endl;
        this_thread::sleep_for(after);
        task();
    },
    [_target = 4.0s]() mutable {
        _target -= 1.0s;
        return _target;
    },
    [] { cout << "done!\n"; }
);
#endif

In [ ]:
namespace v4 {
    
    
template <class S, class T, class F>
void on_expiration_(S scheduler, T timer, F task) {
    auto remaining = timer();
    if (decltype(remaining){0} < remaining) {
        scheduler(remaining, [=] { on_expiration_(scheduler, timer, task); });
    } else {
        task();
    }
}

template <class S, class T, class F>
void on_expiration(S scheduler, T timer, F task) {
    scheduler(timer(), [=] { on_expiration_(scheduler, timer, task); });
}

} // namespace v4

In [ ]:
v4::on_expiration(
    [](auto after, auto task) {
        cout << "try again in " << duration_cast<duration<double>>(after).count() << " seconds." << endl;
        this_thread::sleep_for(after);
        task();
    },
    [_target = 4.0s]() mutable {
        _target -= 1.0s;
        return _target;
    },
    [] { cout << "done!\n"; }
);

In [ ]:
namespace {
    std::map<std::string, int> c = { { "Hello", 42 } };
}

\begin{align}
a * b & = c \\
a & = c / b
\end{align}

In [ ]:
c

In [ ]:
c["Hello"]

In [ ]:
namespace v1 {

template <class T>
class registry {
    unordered_map<size_t, T> _map;
    size_t _id = 0;

public:
    auto append(T element) -> size_t {
        _map.emplace(_id, move(element));
        return _id++;
    }

    void erase(size_t id) { _map.erase(id); }

    auto find(size_t id) -> T* {
        auto p = _map.find(id);
        return (p == end(_map)) ? nullptr : &p->second;
    }

    template <typename F>
    void for_each(F f) const {
        for (const auto& e : _map)
            f(e.second);
    }
};

} // namespace v1

In [ ]:
namespace v2 {

template <class T>
class registry {
    vector<pair<size_t, optional<T>>> _map;
    size_t _size = 0;
    size_t _id = 0;

public:
    //...
    auto append(T element) -> size_t {
        _map.emplace_back(_id, move(element));
        ++_size;
        return _id++;
    }
    //...

    void erase(size_t id) {
        auto p = lower_bound(
            begin(_map), end(_map), id,
            [](const auto& a, const auto& b) { return a.first < b; });
        
        if (p == end(_map) || p->first != id) return;
        
        p->second.reset();
        --_size;
        
        if (_size < (_map.size() / 2)) {
            _map.erase(remove_if(begin(_map), end(_map),
                                 [](const auto& e) { return !e.second; }),
                       end(_map));
        }
    }
    //...

    auto find(size_t id) -> T* {
        auto p = lower_bound(
            begin(_map), end(_map), id,
            [](const auto& a, const auto& b) { return a.first < b; });
        return (p == end(_map) || (p->first != id) || !p->second) ? nullptr :
                                                                    &*p->second;
    }

    template <typename F>
    void for_each(F f) {
        for (const auto& e : _map) {
            if (e.second) f(*e.second);
        }
    }
};

} // namespace v2

In [ ]:
{
    
    int a[] = { 0, 1, 2, 3 };
    int* p = &a[0], * f = &a[2];
    
    std::rotate(p, f, f + 1);
}

In [ ]:
namespace adobe {
template <class T>
class forest {};
} // namespace adobe

namespace v1 {

class view {
    std::list<std::shared_ptr<view>> _children;
    std::weak_ptr<view> _parent;
    //...
};

view screen;

} // namespace v1

namespace v2 {

class view {
    //...
};

class adobe::forest<view> views;

} // namespace v2

In [3]:
struct type {
    void member() { }
};

{
    type* p = nullptr;
    p->member();
    
    if (p) p->member();
}

input_line_9:1:8: error: redefinition of 'type'
struct type {
       ^
input_line_8:1:8: note: previous definition is here
struct type {
       ^


Interpreter Error: 

In [ ]:
namespace {
void f(type* p) {
    //...
    if (p) p->member();
    //...
}

void f(type& p) {
    //...
    p.member();
    //...
}
} // namespace